In [72]:
import os
import random
import string

import datasets
import transformers
import pandas as pd

import gadgets

In [73]:
ds = datasets.load_dataset("gsm8k", "main")

In [74]:
assert set(ds.keys()) == {"train", "test"}

In [75]:
id_chars = list(string.ascii_letters + string.digits)

In [77]:
calc = gadgets.gadget.Calculator()

col_order = ["id", "question", "chain", "result", "result_float"]

df = {}
for split in ds.keys():
    random.seed(f"gsm8k/{split}")
    df[split] = ds[split].to_pandas()
    df[split]["id"] = "gsm8k__" + pd.Series(["".join(random.choices(id_chars, k=16)) for _ in range(len(df[split]))])
    examples = df[split].apply(gadgets.gsm8k.parse, axis=1)
    df[split].drop(columns=["answer"], inplace=True)
    df[split]["chain"] = examples.apply(lambda x: gadgets.markup.to_model_markup(example=x)).apply(str)
    df[split]["result"] = examples.apply(lambda x: x.result)
    df[split]["result_float"] = df[split]["result"].apply(calc._float_eval)
    df[split] = df[split][col_order]

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")


In [78]:
pd.concat(df.values())["id"].is_unique

True

In [79]:
ds_orig_split = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df["train"]),
    "test": datasets.Dataset.from_pandas(df["test"]),
})

#ds_orig_split.push_to_hub("MU-NLPC/Calc-gsm8k", config_name="original-splits")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [85]:
datasets.load_dataset("MU-NLPC/Calc-gsm8k", "original-splits")

{'id': 'gsm8k__78q96B0Sp8u3aEJj',
 'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'chain': 'Natalia sold 48/2 = \n<gadget id="calculator">48/2</gadget>\n<output>24</output>\n24 clips in May.\nNatalia sold 48+24 = \n<gadget id="calculator">48+24</gadget>\n<output>72</output>\n72 clips altogether in April and May.\n\n<result>72</result>',
 'result': '72',
 'result_float': 72.0}